<a href="https://colab.research.google.com/github/neerajthandayan/Deep_Learning/blob/main/Convolutional%20Neural%20Networks/CNN_%2B_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network + GridSearch Cross-validation

#### Part - 1: Data Import & Pre-processing 

In [1]:
# Data Import
from keras.datasets import mnist
from keras.utils import to_categorical  

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

#### Part - 2: Initialising Function for CNN

In [2]:
def cnn_model(act_funct='relu',optimizer='rmsprop',nodes=32,loss='categorical_crossentropy'):
  # Importing Libraries
  from keras import layers
  from keras import models

  #Initialising Convolutional Layer
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation=act_funct, input_shape=(28, 28, 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(nodes, (3, 3), activation=act_funct))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(nodes, (3, 3), activation=act_funct))
  model.summary()
  
  # Adding Neural Net Layer
  model.add(layers.Flatten())
  model.add(layers.Dense(nodes, activation=act_funct))
  model.add(layers.Dense(10, activation='softmax'))
  model.summary()

  # Compiling & Fitting Model
  model.compile(optimizer=optimizer, 
              loss=loss,
              metrics=['accuracy'])
  
  return model

#### Part - 3: Setting up GridSearch

In [ ]:
# GridSearch Cross-validation
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

act_funct = ['relu','elu','selu']
optimizer = ['rmsprop','SGD','adam']
nodes = [64,128]
loss = ['categorical_crossentropy','hinge','squared_hinge']

param_grid = dict(act_funct=act_funct,optimizer=optimizer,nodes=nodes)
Classifier = KerasClassifier(build_fn=cnn_model,epochs=25, batch_size=64,validation_split=0.2)

grid = GridSearchCV(estimator=Classifier,param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_images, train_labels)

#### Part - 4: Cross-validation Summary

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))